Difference between correct and incorrect mean mean heatmaps
    highlights the major differences between correct and incorrect predictions
    red - where correct >> incorrect
    blue - where correct << incorrect 

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
def load_heatmap(image_path):
    """
    Loads and normalizes a heatmap image.
    """
    heatmap = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    if heatmap is None:
        print(f"Error: Unable to load image {image_path}")
        return None
    
    # Convert to float and normalize to [0, 1]
    heatmap = heatmap.astype(np.float32) / 255.0
    return heatmap

In [3]:
def calculate_intersection_heatmap(correct_heatmap_path, incorrect_heatmap_path, output_path):
    """
    Calculates the intersection heatmap and saves it.
    """
    # Load the correct and incorrect mean heatmaps
    correct_heatmap = load_heatmap(correct_heatmap_path)
    incorrect_heatmap = load_heatmap(incorrect_heatmap_path)

    if correct_heatmap is None or incorrect_heatmap is None:
        print("Error: One or both heatmaps could not be loaded.")
        return

    # Ensure both heatmaps have the same shape
    if correct_heatmap.shape != incorrect_heatmap.shape:
        print(f"Resizing incorrect heatmap from {incorrect_heatmap.shape} to {correct_heatmap.shape}")
        incorrect_heatmap = cv2.resize(incorrect_heatmap, (correct_heatmap.shape[1], correct_heatmap.shape[0]))

    # Intersection calculation (element-wise minimum)
    intersection_heatmap = np.minimum(correct_heatmap, incorrect_heatmap)

    # Normalize the intersection map to [0, 1] for visualization
    inter_min, inter_max = np.min(intersection_heatmap), np.max(intersection_heatmap)
    intersection_heatmap = (intersection_heatmap - inter_min) / (inter_max - inter_min + 1e-8)

    # Convert to uint8 and apply JET colormap
    intersection_heatmap_uint8 = np.uint8(255 * intersection_heatmap)
    colored_intersection_heatmap = cv2.applyColorMap(intersection_heatmap_uint8, cv2.COLORMAP_JET)

    # Save the intersection heatmap
    cv2.imwrite(output_path, colored_intersection_heatmap)
    print(f"Intersection heatmap saved at: {output_path}")


In [4]:

def main():
    structures = ["abdomen", "femur", "head"] 
    birth_types = ["ces", "vag"]
    datasets = ["train", "test"]
    confidences = ["high", "low"]

    base_dir = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted"
    mean_folder = os.path.join(base_dir, "mean_mean_heatmaps")
    output_folder = os.path.join(base_dir, "INTERSECTION_MEAN_HEATMAPS")
    os.makedirs(output_folder, exist_ok=True)

    for structure in structures:
        for birth_type in birth_types:
            for dataset in datasets:
                for confidence in confidences:
                    correct_file = f"{structure}_{birth_type}_{dataset}_correct_{confidence}_confidence_mean_heatmap.png"
                    incorrect_file = f"{structure}_{birth_type}_{dataset}_incorrect_{confidence}_confidence_mean_heatmap.png"
                    output_file = f"{structure}_{birth_type}_{dataset}_{confidence}_confidence_intersection_heatmap.png"

                    correct_path = os.path.join(mean_folder, correct_file)
                    incorrect_path = os.path.join(mean_folder, incorrect_file)
                    output_path = os.path.join(output_folder, output_file)

                    if os.path.exists(correct_path) and os.path.exists(incorrect_path):
                        print(f"[INTERSECTING] {structure}, {birth_type}, {dataset}, {confidence}")
                        calculate_intersection_heatmap(correct_path, incorrect_path, output_path)
                    else:
                        print(f"[SKIPPED] Missing one of the heatmaps for: {structure}, {birth_type}, {dataset}, {confidence}")

if __name__ == "__main__":
    main()


[SKIPPED] Missing one of the heatmaps for: abdomen, ces, train, high
[INTERSECTING] abdomen, ces, train, low
Intersection heatmap saved at: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted\INTERSECTION_MEAN_HEATMAPS\abdomen_ces_train_low_confidence_intersection_heatmap.png
[INTERSECTING] abdomen, ces, test, high
Intersection heatmap saved at: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted\INTERSECTION_MEAN_HEATMAPS\abdomen_ces_test_high_confidence_intersection_heatmap.png
[INTERSECTING] abdomen, ces, test, low
Intersection heatmap saved at: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted\INTERSECTION_MEAN_HEATMAPS\abdomen_ces_test_low_confidence_intersection_heatmap.png
[INTERSECTING] abdomen, vag, train, high
Intersection heatmap saved at: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESU